In [1]:
from ase.atoms import Atoms
from ase.io import write
from ase.visualize import view
import jax
import jax.numpy as jnp
import sys
sys.path.append('..')
import analysis
import datatypes
from input_pipeline_tf import get_datasets
from models import get_first_node_indices
from train import get_predictions

2023-03-06 22:48:56.598966: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-06 22:48:58.783502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-06 22:48:58.783682: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-06 22:48:58.783695: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:
config, best_state, metrics_for_best_state = analysis.load_from_workdir('../workdirs/mace0')

/home/songk/atomicarchitects/spherical_harmonic_net/qm9_data_tf/
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-06 22:49:03.794562: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-06 22:49:03.794733: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-06 22:49:03.794806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-06 22:49:03.794866: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-03-06 22:49:03.794926: W tensorfl

In [3]:
cutoff = 5.0
key = jax.random.PRNGKey(0)
epsilon = 1e-4

qm9_datasets = get_datasets(key, config)
# atoms = qm9_data[1]

/home/songk/atomicarchitects/spherical_harmonic_net/qm9_data_tf/


In [4]:
example_graph = next(qm9_datasets["test"].as_numpy_iterator())
frag = datatypes.Fragment.from_graphstuple(example_graph)
frag = jax.tree_map(jnp.asarray, frag)

In [5]:
atomic_numbers = jnp.array([1, 6, 7, 8, 9])
numbers_to_symbols = {1: 'H', 6: 'C', 7: 'N', 8: 'O', 9: 'F'}

def get_numbers(species: jnp.ndarray):
    numbers = []
    for i in species:
        numbers.append(atomic_numbers[i])
    return jnp.array(numbers)

In [6]:
preds = get_predictions(best_state, frag)

In [7]:
true_focus = get_first_node_indices(frag)
# this is what the current setup is using as the foci anyway; should be changed later

In [101]:
positions_split = frag.nodes.positions.split(true_focus[1:], axis=0)
species_split = frag.nodes.species.split(true_focus[1:], axis=0)

element_radii = [32 * 0.01, 75 * 0.01, 71 * 0.01, 63 * 0.01, 64 * 0.01]

In [114]:
ndx = 14
frag_atoms = Atoms(positions=positions_split[ndx].tolist(), numbers=get_numbers(species_split[ndx].tolist()))
v = view(frag_atoms, viewer='ngl')

sphere = v.view.shape.add_sphere(
    positions_split[ndx][0].tolist(),
    [1, 0.85, 0],
    element_radii[species_split[ndx].tolist()[0]] * 0.6,
    "yellow sphere with 0.5A radius",
)
v.view.update_representation(component=1, opacity=0.5)

In [115]:
v

In [ ]:
# save to image files instead:
for i, frag_to_write in enumerate(frags):
    write(f'image{i}.png', Atoms(positions=frag_to_write.nodes.positions, numbers=get_numbers(frag_to_write.nodes.species)))